In [ ]:
import boto3
import pandas as pd
from io import BytesIO
import os
from datetime import datetime
import re
import pyarrow as pa
from dotenv import load_dotenv
import mysql.connector
from mysql.connector import connection

In [ ]:
df = pd.read_csv(r"C:\Users\documentos-estruturados\list.csv")
df = df[['Id']]
df = df.drop_duplicates(subset=['Id'])

In [ ]:
df_contagem = pd.read_csv(r"C:\Users\documentos-estruturados\list.csv")
df_contagem = df_contagem[['Id']]
contagem = df_contagem[['Id']].value_counts().reset_index()
contagem.columns = ['Id', 'QuantidadeDocumentos']
df_contagem = df_contagem.merge(contagem, on='Id', how='left')
df_contagem = df_contagem.drop_duplicates(subset='Id')
df_contagem.to_excel("contagem.xlsx")

In [ ]:
#conexao banco

conn = connection.MySQLConnection(user='x', password ='x', host ='x', database = " x")
cursor = conn.cursor()


In [ ]:

# Lista para armazenar os resultados das consultas exames

resultados = []

# Iterar pelos IDs no DataFrame
for _, row in df.iterrows():
    id_procurado = row['Id']
    comando = f"query para localizar os dados procurados = {id_procurado}"
    cursor.execute(comando)
    
    # Executar a consulta SQL e carregar os resultados em um DataFrame
    df_resultado = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
    
    # Verifique se há resultados e os adicione à lista
    if not df_resultado.empty:
        resultados.append(df_resultado)



In [ ]:
# Concatenar os resultados em um único DataFrame, se necessário
if resultados:
    df_exames = pd.concat(resultados, ignore_index=True)
    print(df_exames)
else:
    print("Nenhum resultado encontrado para os IDs fornecidos.")

In [ ]:
#lista de atestados
resultados = []

# Iterar pelos IDs no DataFrame
for _, row in df.iterrows():
    id_procurado = row['Id']
    comando = f"query para localizar os dados procurados  = {id_procurado}"
    cursor.execute(comando)
    
    # Executar a consulta SQL e carregar os resultados em um DataFrame
    df_resultado = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
    
    # Verifique se há resultados e os adicione à lista
    if not df_resultado.empty:
        resultados.append(df_resultado)

In [ ]:
# Concatenar os resultados em um único DataFrame, se necessário
if resultados:
    df_atestado = pd.concat(resultados, ignore_index=True)
    print(df_atestado)
else:
    print("Nenhum resultado encontrado para os IDs fornecidos.")

In [ ]:
#lista de medicamentos
resultados = []

# Iterar pelos IDs no DataFrame
for _, row in df.iterrows():
    id_procurado = row['Id']
    comando = f"query para localizar os dados procurados  = {id_procurado}"
    cursor.execute(comando)
    
    # Executar a consulta SQL e carregar os resultados em um DataFrame
    df_resultado = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
    
    # Verifique se há resultados e os adicione à lista
    if not df_resultado.empty:
        resultados.append(df_resultado)

In [ ]:
# Concatenar os resultados em um único DataFrame, se necessário 
if resultados:
    df_prescricoes = pd.concat(resultados, ignore_index=True)
    print(df_prescricoes)
else:
    print("Nenhum resultado encontrado para os IDs fornecidos.")

In [ ]:
df_atestado['TipoDocumento'] = 'Atestado'
df_exames['TipoDocumento'] = 'Exame'
df_prescricoes['TipoDocumento'] = 'Medicamentos'

In [ ]:
load_dotenv()
config = {
    "bucket": os.getenv('bucket'),
    "path": os.getenv('path')
}

In [ ]:
#login para acessar aws
from decouple import config

aws_access_key_id = os.environ['AWS_ACCESS_KEY_ID']
aws_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY']



s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

In [ ]:
import boto3

def list_objects_in_bucket(bucket_name):
    s3_client = boto3.client('s3')
    response = s3_client.list_objects(Bucket=bucket_name)
    
    print(f'Objetos no bucket {bucket_name}:')
    for obj in response.get('Contents', []):
        print(obj['Key'])

# Substitua 'nome_do_seu_bucket' pelo nome do bucket que você deseja listar.
list_objects_in_bucket('bucket')


In [ ]:
# Prefixo do diretório que você deseja encontrar - aqui só retorna os 1000 primeiros
bucket_origem= 'bucket_origem'
diretorio_origem = 'diretorio_origem/'

# Lista os objetos no bucket
response = s3.list_objects_v2(Bucket=bucket_origem)

# Itera sobre os objetos encontrados
for obj in response.get('Contents', []):
    objeto_key = obj['Key']
    
    # Verifica se o objeto é um diretório procurado
    if objeto_key.startswith(diretorio_origem):
        print(f"Diretório encontrado: {objeto_key}")

In [ ]:
#retornar todos
def list_all_objects(bucket_name, prefix):
    s3_client = boto3.client('s3')
    continuation_token = None
    all_files = []
    while True:
        list_kwargs = {
            'Bucket': bucket_name,
            'Prefix': prefix,
        }
        # Se temos um continuation token, adicionamos ao pedido
        if continuation_token:
            list_kwargs['ContinuationToken'] = continuation_token
        # Fazemos o pedido para listar os objetos
        response = s3_client.list_objects_v2(**list_kwargs)
        # Adicionamos os objetos retornados à nossa lista
        all_files.extend(response.get('Contents', []))
        # Se não houver mais objetos, paramos o loop
        if not response.get('IsTruncated'):
            break
        # Senão, atualizamos o continuation token para a próxima iteração
        continuation_token = response.get('NextContinuationToken')
    return [file['Key'] for file in all_files]
# Uso
bucket_name = 'bucket_name'
prefix = 'prefix/'
files = list_all_objects(bucket_name, prefix)
print(files)

In [ ]:
import logging
import boto3
from botocore.exceptions import ClientError


def create_presigned_url(bucket_name, object_name, expiration=259200):

    # Generate a presigned URL for the S3 object
    s3_client = boto3.client('s3')
    try:
        response = s3_client.generate_presigned_url('get_object',
                                                    Params={'Bucket': bucket_name,
                                                            'Key': object_name},
                                                    ExpiresIn=expiration)
    except ClientError as e:
        logging.error(e)
        return None

    # The response contains the presigned URL
    return response

In [ ]:
#exames

if resultados:
    
    # Adicionar a coluna 'x' com URLs assinadas
    df_exames['Pre_signed'] = df_exames.apply(lambda row: create_presigned_url(bucket_name, 'Exame/' + row['URL']), axis=1)
    
    
    print("DataFrame salvo com URLs assinadas.")
else:
    print("Nenhum resultado encontrado para os IDs fornecidos.")

In [ ]:
#atestado
if resultados:
    
    # Adicionar a coluna 'x' com URLs assinadas
    df_atestado['Pre_signed'] = df_atestado.apply(lambda row: create_presigned_url(bucket_name, 'Atestado/' + row['URL']), axis=1)
    
    
    print("DataFrame salvo com URLs assinadas.")
else:
    print("Nenhum resultado encontrado para os IDs fornecidos.")

In [ ]:
#prescricoes

if resultados:
    
    # Adicionar a coluna 'x' com URLs assinadas
    df_prescricoes['Pre_signed'] = df_prescricoes.apply(lambda row: create_presigned_url(bucket_name, 'Prescricao/' + row['URL']), axis=1)
    
    
    print("DataFrame salvo com URLs assinadas.")
else:
    print("Nenhum resultado encontrado para os IDs fornecidos.")

In [ ]:
dataframes = [df_laudo, df_encaminhamento, df_atestado, df_exames,df_prescricoes]
df_final = pd.concat(dataframes, ignore_index=True)
df_final = df_final[['idPrescricao','TipoDocumento','Pre_signed']]
df_final = df_final.rename(columns={'idPrescricao': 'Id', 'Pre_signed': 'DocURL '})
df_final.to_csv("Prescricoes.csv",index=False)